In [1]:
import geopandas as gpd
import pandas as pd
import numpy
import fiona

In [2]:
pplants = gpd.read_file(r'.\data\PowerPlants_US_EIA\PowerPlants_US_202001.shp')

In [3]:
pplants['combine'] = pplants[['Total_MW','Coal_MW','NG_MW','Crude_MW','Bio_MW','Hydro_MW','HydroPS_MW','Nuclear_MW','Solar_MW','Wind_MW','Geo_MW','Bat_MW','Other_MW']].values.tolist()

In [4]:
pplants.head(3)

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,StateName,Zip,Street_Add,...,Bat_MW,Other_MW,source_des,tech_desc,Source,Period,Latitude,Longitude,geometry,combine
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,Alabama,35476,19001 Lock 17 Road,...,None,None,Hydroelectric = 53 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202001,33.458665,-87.356823,POINT (-87.35682 33.45867),"[53.0, 0.0, 0.0, 0.0, None, 53.0, 0.0, 0.0, 0...."
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,Alabama,36512,North Highway 43,...,None,None,"Coal = 1118.5 MW, Natural Gas = 1268.4 MW",Conventional Steam Coal; Natural Gas Fired Com...,"EIA-860, EIA-860M and EIA-923",202001,31.006900,-88.010300,POINT (-88.01030 31.00690),"[2386.9, 1118.5, 1268.4, 0.0, None, 0.0, 0.0, ..."
2,4,Walter Bouldin Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,Alabama,36092,750 Bouldin Dam Road,...,None,None,Hydroelectric = 224.1 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202001,32.583889,-86.283056,POINT (-86.28306 32.58389),"[224.1, 0.0, 0.0, 0.0, None, 224.1, 0.0, 0.0, ..."


In [38]:
pplants["source_des_noMW"] = pplants['source_des'].replace({' MW': ''}, regex=True)

In [39]:
pplants["source_des_noMW"] = pplants['source_des'].replace({', ': ','}, regex=True)

In [40]:
pplants['fuel'] = pplants["source_des_noMW"].str.split(',',expand=False) 

In [41]:
pplants.head(3)

,code,plant_name,plant_name,Utility_ID,sector_nam,City,County,StateName,Zip,Street_Add,...,Source,Period,Latitude,Longitude,geometry,combine,source_des_noMW,fuel,fueldict,fuel_source
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,Alabama,35476,19001 Lock 17 Road,...,"EIA-860, EIA-860M and EIA-923",202001,33.458665,-87.356823,POINT (-87.35682 33.45867),"[53.0, 0.0, 0.0, 0.0, None, 53.0, 0.0, 0.0, 0....",Hydroelectric = 53 MW,[Hydroelectric = 53 MW],{'Hydroelectric': '53'},{'Hydroelectric': '53'}
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,Alabama,36512,North Highway 43,...,"EIA-860, EIA-860M and EIA-923",202001,31.006900,-88.010300,POINT (-88.01030 31.00690),"[2386.9, 1118.5, 1268.4, 0.0, None, 0.0, 0.0, ...","Coal = 1118.5 MW,Natural Gas = 1268.4 MW","[Coal = 1118.5 MW, Natural Gas = 1268.4 MW]","{'Coal': '1118.5', ' Natural Gas': '1268.4'}","{'Coal': '1118.5', ' Natural Gas': '1268.4'}"
2,4,Walter Bouldin Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,Alabama,36092,750 Bouldin Dam Road,...,"EIA-860, EIA-860M and EIA-923",202001,32.583889,-86.283056,POINT (-86.28306 32.58389),"[224.1, 0.0, 0.0, 0.0, None, 224.1, 0.0, 0.0, ...",Hydroelectric = 224.1 MW,[Hydroelectric = 224.1 MW],{'Hydroelectric': '224.1'},{'Hydroelectric': '224.1'}


In [42]:
pplants["fuel_source"] = pplants['source_des_noMW'].apply(lambda x: dict(kv.split(' = ') for kv in x.split(',')))


In [43]:
pplants.rename({'Plant_Code': 'code', 'Plant_Name': 'plant_name','Total_MW': 'capacity_mw'}, axis=1, inplace=True)

In [44]:
pplants.head(2)

,code,plant_name,plant_name,Utility_ID,sector_nam,City,County,StateName,Zip,Street_Add,...,Source,Period,Latitude,Longitude,geometry,combine,source_des_noMW,fuel,fueldict,fuel_source
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,Alabama,35476,19001 Lock 17 Road,...,"EIA-860, EIA-860M and EIA-923",202001,33.458665,-87.356823,POINT (-87.35682 33.45867),"[53.0, 0.0, 0.0, 0.0, None, 53.0, 0.0, 0.0, 0....",Hydroelectric = 53 MW,[Hydroelectric = 53 MW],{'Hydroelectric': '53'},{'Hydroelectric': '53 MW'}
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,Alabama,36512,North Highway 43,...,"EIA-860, EIA-860M and EIA-923",202001,31.006900,-88.010300,POINT (-88.01030 31.00690),"[2386.9, 1118.5, 1268.4, 0.0, None, 0.0, 0.0, ...","Coal = 1118.5 MW,Natural Gas = 1268.4 MW","[Coal = 1118.5 MW, Natural Gas = 1268.4 MW]","{'Coal': '1118.5', ' Natural Gas': '1268.4'}","{'Coal': '1118.5 MW', 'Natural Gas': '1268.4 MW'}"


In [45]:
pplants[['code','plant_name','capacity_mw','fuel_source','geometry']].to_file("..\\data\\plants.geojson", driver='GeoJSON')